In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
from tqdm import tqdm
from scipy.integrate import simpson
import sys
sys.path.append("..")


FORCES = ["LO", "NLO", "N2LO", "N3LO", "N4LO", "N4LO+"]
CUTOFF = [400, 450, 500, 550]

# plt.style.use(['science', 'retro'])

mpl.rcParams['mathtext.fontset'] = 'custom'
mpl.rcParams['mathtext.rm'] = 'DejaVu Sans'
mpl.rcParams['mathtext.it'] = 'DejaVu Sans:italic'
mpl.rcParams['mathtext.bf'] = 'DejaVu Sans:bold'

plt.rcParams["font.family"] = "sans-serif"
plt.rcParams["font.sans-serif"] = "DejaVu Sans"

plt.rc('text', usetex=True)
# plt.rc('text', usetex=False)
plt.rcParams['axes.linewidth'] = 1.2
mpl.rcParams['lines.linewidth'] = 2

line_styles = {"LO": {"marker": "d", "markevery": 10, "markersize": 3, "color": "tab:cyan"},
               "NLO": {"marker": ".", "markevery": 10, "color": "orange"},
               "N2LO": {"ls":  (0, (5, 3)), "color": "green"},
               "N3LO": {"ls": "dotted", "color": "blue"},
               "N4LO": {"ls": (0, (5, 2, 0.8, 2, 0.8, 2)), "color": "red"},
               "N4LO+": {"color": "black"},
               "AV18": {"ls": (0, (5, 2, 1.3, 2,)), "color": "blueviolet"},
               "SIEGERT": {"color": "blue"},
               "1NC": {"color": "deeppink", "ls": "dashed"},
               "PW": {"ls":  (0, (5, 2, 1.3, 2,)), "color": "green"}
               }
ls_cutoff = {400: line_styles["N4LO"],
             450: line_styles["N4LO+"],
             500: line_styles["N2LO"],
             550: line_styles["N3LO"],
             }

exp_style = [
    {"fmt": "ok"},
    {"fmt": "ok", "markerfacecolor": "None", "markeredgecolor": "k"},
    {"fmt": "sk"},
    {"fmt": "^k"},
    {"fmt": "Xk"},
    {"fmt": "dk"},
]

trunc_colors = {
    "NLO": "bisque",
    "N2LO": "limegreen",
    "N3LO": "dodgerblue",
    "N4LO": "red",
    "N4LO+": "black",
}

marker_common = {"markersize": 6, "elinewidth": 0.9, "capsize": 2.5}


df_my = pd.read_csv("deuteron_all_data.csv")
df_my.T21D2 *= 2
df_my.T22D2 *= 2
df_my.T21D *= 2
df_my.T22D *= 2
df_my.AX *= -1
df_my.AX2 *= -1
df_my.T11D2 *= -2
df_my.T11D *= -2
df_exp = pd.read_csv("deuteron_exp_diffcross.csv")
df_exp_tot = pd.read_csv("deuteron_exp_totcross.csv")

OBSERVABLES = [ob for ob in df_my.columns.values if ob not in [
    "THCM", "angle", "CUTOFF", "FORCE"]]

my_energies = df_my.Energy.unique()
my_energies.sort()

rachek_bins = pd.read_csv("ExpData/TensoeAnalyzingPower/rachek_t2i_vs_theta.txt", delim_whitespace=True,
                          names=["Emin", "Emax", "Emean", "THmin", "THmax", "THmean",
                                 "T20", "T20stater", "T20syser", "T21", "T21stater", "T21syser",
                                 "T22", "T22stater", "T22syser"])


FileNotFoundError: [Errno 2] No such file or directory: 'deuteron_all_data.csv'

In [ ]:
df_tmp = df_my[(df_my.CUTOFF == 450) &
            (df_my.Energy == 100) &
            (df_my.FORCE == "N4LO+") &
            (df_my.WAVE.isin(["1NC", "SIEGERT"]))]
df_tmp.sort_values("angle", inplace=True)

In [ ]:
fig = plt.figure(figsize=(6, 4))
gs = fig.add_gridspec(nrows=2, ncols=2, hspace=0)
axes = gs.subplots(sharex=True)
fig.supxlabel("$\Theta_\mathrm{p}$ [deg]", y=0.02, size=12)
# ylabel = obs
# fig.supylabel(ylabel, x=0.03, size=16)
for obs, lab, ax in zip(["T20D", "T21D", "T22D", "T11D"],
                    ["T$_{20}$", "T$_{21}$", "T$_{22}$", "iT$_{11}$"],
                    axes.flatten()):
    ax.plot(df_tmp[(df_tmp.WAVE == "SIEGERT")].angle,
                df_tmp[(df_tmp.WAVE == "SIEGERT")][obs+"2"],
                **line_styles["SIEGERT"], label="Full")
    ax.plot(df_tmp[(df_tmp.WAVE == "1NC")].angle,
            df_tmp[(df_tmp.WAVE == "1NC")][obs+"2"],
                **line_styles["1NC"], label="1NC")
    # axes[i].plot(df_tmp[(df_tmp.WAVE == "1NC")].angle, df_tmp[(df_tmp.WAVE == "1NC")].CROSS)
    ax.plot(df_tmp[(df_tmp.WAVE == "SIEGERT")].angle,
                   df_tmp[(df_tmp.WAVE == "SIEGERT")][obs],
                **line_styles["PW"], label="PW")
    
    ax.set_ylabel(lab)

axes[0,0].legend()
for j, ax in enumerate(axes.flatten()):
    # style
    # tmp
    # if j == 1:
    #     ax.legend(fontsize=9, handlelength=3, loc="upper center")
    # else:
    #     ax.legend(fontsize=9, handlelength=3, loc="best")
    # ax.legend(fontsize=9, handlelength=3, loc="best")
    
    ax.xaxis.set_minor_locator(mpl.ticker.AutoMinorLocator())
    ax.yaxis.set_minor_locator(mpl.ticker.AutoMinorLocator())
    ax.tick_params(which='minor', direction='in', length=4,
                width=0.5, top=True, right=True)
    ax.tick_params(which='major', direction='in',
                length=7, width=1, top=True, right=True, zorder=1)
    ax.set_xticks(np.arange(0, 180+j-1, 30))
    ax.set_xlim([0, 180])
    # ax.grid(alpha=0.2)
    ax.set_zorder(-1)
fig.tight_layout()
# plt.savefig(f"../Figures_De/TensorPowers_100mev_pw_1nc.pdf",
#                     dpi=600, facecolor="white")